In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
#loading in the csv data
df = pd.read_csv('/kaggle/input/fashion-product-images-small/myntradataset/styles.csv',error_bad_lines=False)

df.head()

In [ ]:
df[1748:1754].head(6)

In [ ]:
print(df[540:541].id.values[0])

In [ ]:
from matplotlib import pyplot as plt
import cv2
for i in range(1748, 1754):
    
    thisId = str(df[i:i+1].id.values[0])
    
    imageName = '/kaggle/input/fashion-product-images-small/myntradataset/images/'+ thisId +'.jpg'
    image = cv2.imread(imageName)
    image = RGB_im = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.imshow(image)
    plt.title(f'Image {thisId}')
    plt.show()

In [ ]:
df = df.dropna()
df.nunique()
df.columns

In [ ]:
# Looking at all the unique labels in all categorical columns 
cat_columns = ['gender', 'masterCategory', 'subCategory', 'articleType','baseColour', 'season', 'year', 'usage']

for col in cat_columns:
    print(col)
    print(df[col].unique())
    print('-------------------------')

The images in this Dataset are very low resolution (80x60). We will be using the categories that are visually distinct even at such a low resolution.

The categories year, usage, season, and gender mighht not be clearly visually distinct in some cases, so we wont be using them.

The categories, masterCategory and subCategory are distinct enough groups, but they are not specific enough for practical use. 


In [ ]:
value_counts = df['subCategory'].value_counts()

indexes = value_counts.index

values = value_counts.values

types_used = indexes[:i]
print('Types used: ',types_used)

In [ ]:
# Taking a subset of 20000 images
old_df = df
df = old_df[:25000]
len(df)

In [ ]:
df_trimmed = old_df[:15923]

df_extended = pd.concat([old_df, df_trimmed], ignore_index=True)

len(df_extended)

Now we will load in all the images from the remaining rows, and convert them to numpy arrays with img_to_array function in keras.

In [ ]:
# AUGMENT

data = []

# Reading all the images and processing the data in them 

from tensorflow.keras.preprocessing.image import img_to_array
import cv2

IX = 80
IY = 60

invalid_ids = []

count = 0

for name in df.id:

    try:
        count += 1
        image = cv2.imread('/kaggle/input/fashion-product-images-small/myntradataset/images/'+str(name)+'.jpg')
        imageResized = cv2.resize(image, (IX,IY) )
        
        # Flipping the image horizontally -> augmentation of the first 15923 image starts here
        if (count >= 44077):
            imageResized = cv2.flip(imageResized, 1)
        
        image = img_to_array(imageResized)
        data.append(image)        
    except: 
        # Images for certain ids are missing, so they are not added to the dataset  
        invalid_ids.append(name)

In [ ]:
# NO AUGMENT

#data = []

# Reading all the images and processing the data in them 

#from tensorflow.keras.preprocessing.image import img_to_array
#import cv2

#IX = 80
#IY = 60

#invalid_ids = []

#for name in df.id:

#    try:
#        image = cv2.imread('/kaggle/input/fashion-product-images-small/myntradataset/images/'+str(name)+'.jpg')
#        imageResized = cv2.resize(image, (IX,IY) )
#        image = img_to_array(imageResized)
#        data.append(image)        
#    except: 
        # Images for certain ids are missing, so they are not added to the dataset  
#        invalid_ids.append(name)

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
import cv2

IX = 80
IY = 60

In [ ]:
labelsInit = []

count = 0

for index, row in df.iterrows():
    
    count += 1
    
    if row['id'] in invalid_ids:
        continue

    labelsInit.append(row['subCategory'])
    
print(count)
len(labelsInit)

In [ ]:
# Unique categories
df["subCategory"].unique()

In [ ]:
import numpy as np

# converting data into numpy arrays

data = np.array(data, dtype="float") / 255.0
labels = np.array(labelsInit)

print(labels)

Now we will create binary vectors as the outputs of the model

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# creating a binary vector for the input labels 

mlb = LabelBinarizer()
labels = mlb.fit_transform(labels)

print(mlb.classes_)

In [ ]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

inputShape = (IY, IX, 3)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4)) # Dropout increased from 0.3 to 0.4

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4)) # Dropout increased from 0.3 to 0.4

model.add(Flatten()) 

model.add(Dense(128))
model.add(Activation('relu'))


out = len(mlb.classes_)

model.add(Dense(out))
model.add(Activation('sigmoid'))

optimizer = Adam(learning_rate=0.000875, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['mse'])

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

# splitting data into testing and training set 

(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.2, random_state=42)

In [ ]:
batch = 32
E = 100

#training the model 
model.fit(x=trainX,y=trainY,
          epochs=E ,verbose=1)

In [ ]:
# TEST SET
print("Predicting on the test set")
preds = model.predict(testX)


# since the predictions of the model are sigmoid, we will first binarize them to 0 or 1
pred_binarized = []

for pred in preds:
    index = np.argmax(pred)
    ar = np.zeros((pred.shape[0],), dtype=int)
    ar[index] = 1
    pred_binarized.append(ar)

pred_binarized = np.array(pred_binarized)   

# we convert the output vectors to the predicted labels
true_test_labels = mlb.inverse_transform(testY)
pred_test_labels = mlb.inverse_transform(pred_binarized)

correct = 0
wrong = 0

# Evaluating the predictions of the model

for i in range(len(testY)):

    true_labels = list(true_test_labels[i])

    pred_labels = list(pred_test_labels[i])

    label1 = true_labels[0]
    label2 = true_labels[1]

    if label1 in pred_labels:
        correct+=1
    else:
        wrong+=1

    if label2 in pred_labels:
        correct+=1
    else:
        wrong+=1    



print('correct: ', correct)
print('missing/wrong: ', wrong)
print('Accuracy: ',correct/(correct+wrong))

In [ ]:
# TRAIN SET
print("Predicting on the train set")
predsTrain = model.predict(trainX)


# since the predictions of the model are sigmoid, we will first binarize them to 0 or 1
pred_binarized_train = []

for pred in predsTrain:
    index = np.argmax(pred)
    ar = np.zeros((pred.shape[0],), dtype=int)
    ar[index] = 1
    pred_binarized_train.append(ar)

pred_binarized_train = np.array(pred_binarized_train)   

# we convert the output vectors to the predicted labels
true_train_labels = mlb.inverse_transform(trainY)
pred_train_labels = mlb.inverse_transform(pred_binarized_train)

correct = 0
wrong = 0

# Evaluating the predictions of the model

for i in range(len(trainY)):

    true_labels = list(true_train_labels[i])

    pred_labels = list(pred_train_labels[i])

    label1 = true_labels[0]
    label2 = true_labels[1]
    
    if label1 in pred_labels:
        correct+=1
    else:
        wrong+=1

    if label2 in pred_labels:
        correct+=1
    else:
        wrong+=1    

print('correct: ', correct)
print('missing/wrong: ', wrong)
print('Accuracy: ',correct/(correct+wrong))

In [ ]:
for i in range(len(testY)):
    if (true_test_labels[i] != pred_test_labels[i]):
        print('True labels: ',true_test_labels[i],' Predicted labels: ',pred_test_labels[i])